# `dtreeviz` scikit-learn Examples

([View this notebook in Colab](https://colab.research.google.com/github/parrt/dtreeviz/blob/master/notebooks/dtreeviz_sklearn_visualisations.ipynb))

The [dtreeviz](https://github.com/parrt/dtreeviz) library is designed to help machine learning practitioners visualize and interpret decision trees and decision-tree-based models, such as gradient boosting machines.  

The purpose of this notebook is to illustrate the main capabilities and functions of the dtreeviz API. To do that, we will use scikit-learn and the toy but well-known Titanic data set for illustrative purposes.  Currently, dtreeviz supports the following decision tree libraries:

* [scikit-learn](https://scikit-learn.org/stable)
* [XGBoost](https://xgboost.readthedocs.io/en/latest)
* [Spark MLlib](https://spark.apache.org/mllib/)
* [LightGBM](https://lightgbm.readthedocs.io/en/latest/)
* [Tensorflow](https://www.tensorflow.org/decision_forests)

To interopt with these different libraries, dtreeviz uses an adaptor object, obtained from function `dtreeviz.model()`, to extract model information necessary for visualization. Given such an adaptor object, all of the dtreeviz functionality is available to you using the same programmer interface. The basic dtreeviz usage recipe is:

1. Import dtreeviz and your decision tree library
2. Acquire and load data into memory
3. Train a classifier or regressor model using your decision tree library
4. Obtain a dtreeviz adaptor model using<br>`viz_model = dtreeviz.model(your_trained_model,...)`
5. Call dtreeviz functions, such as<br>`viz_model.view()` or `viz_model.explain_prediction_path(sample_x)`

The four categories of dtreeviz functionality are:

1. Tree visualizations
2. Prediction path explanations
3. Leaf information
4. Feature space exploration

We have grouped code examples by [classifiers](#Classifiers) and [regressors](#Regressors), with a follow up section on [partitioning feature space](#Feature-Space-Partitioning).

*These examples require dtreeviz 2.0 or above because the code uses the new API introduced in 2.0.*

## Setup

In [1]:
import sys
import os

In [ ]:
%config InlineBackend.figure_format = 'retina' # Make visualizations look good
#%config InlineBackend.figure_format = 'svg' 
%matplotlib inline

if 'google.colab' in sys.modules:
  !pip install -q dtreeviz

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

import dtreeviz

random_state = 1234 # get reproducible trees

## Load Sample Data

In [ ]:
dataset_url = "https://raw.githubusercontent.com/parrt/dtreeviz/master/data/titanic/titanic.csv"
dataset = pd.read_csv(dataset_url)
# Fill missing values for Age
dataset.fillna({"Age":dataset.Age.mean()}, inplace=True)
# Encode categorical variables
dataset["Sex_label"] = dataset.Sex.astype("category").cat.codes
dataset["Cabin_label"] = dataset.Cabin.astype("category").cat.codes
dataset["Embarked_label"] = dataset.Embarked.astype("category").cat.codes

# Classifiers

To demonstrate classifier decision trees, we trying to model using six features to predict the boolean survived target.

In [ ]:
features = ["Pclass", "Age", "Fare", "Sex_label", "Cabin_label", "Embarked_label"]
target = "Survived"

tree_classifier = DecisionTreeClassifier(max_depth=3, random_state=random_state)
tree_classifier.fit(dataset[features].values, dataset[target].values)

## Initialize dtreeviz model (adaptor)

To adapt dtreeviz to a specific model, use the `model()` function to get an adaptor.  You'll need to provide the model, X/y data, feature names, target name, and target class names:

In [ ]:
viz_model = dtreeviz.model(tree_classifier,
                           X_train=dataset[features], y_train=dataset[target],
                           feature_names=features,
                           target_name=target, class_names=["survive", "perish"])

We'll use this model to demonstrate dtreeviz functionality in the following sections; the code will look the same for any decision tree library once we have this model adaptor.

## Tree structure visualizations

To show the decision tree structure using the default visualization, call `view()`:

In [ ]:
viz_model.view(scale=0.8)

To change the visualization, you can pass parameters, such as changing the orientation to left-to-right:

In [ ]:
viz_model.view(orientation="LR")

To visualize larger trees, you can reduce the amount of detail by turning off the fancy view:

In [ ]:
viz_model.view(fancy=False)

Another way to reduce the visualization size is to specify the tree depths of interest:

In [ ]:
viz_model.view(depth_range_to_display=(1, 2)) # root is level 0

## Prediction path explanations

For interpretation purposes, we often want to understand how a tree behaves for a specific instance. Let's pick a specific instance:

In [ ]:
x = dataset[features].iloc[10]
x

and then display the path through the tree structure:

In [ ]:
viz_model.view(x=x)

In [ ]:
viz_model.view(x=x, show_just_path=True)

You can also get a string representation explaining the comparisons made as an instance is run down the tree:

In [ ]:
print(viz_model.explain_prediction_path(x))

If you'd like the feature importance for a specific instance, as calculated by the underlying decision tree library, use `instance_feature_importance()`:

In [ ]:
viz_model.instance_feature_importance(x, figsize=(3.5,2))

## Leaf info

There are a number of functions to get information about the leaves of the tree.

In [ ]:
viz_model.leaf_sizes(figsize=(3.5,2))

In [ ]:
viz_model.ctree_leaf_distributions(figsize=(3.5,2))

In [ ]:
viz_model.node_stats(node_id=6)

In [ ]:
viz_model.leaf_purity(figsize=(3.5,2))

# Regressors

To demonstrate regressor tree visualization, we start by creating a regressors model that predicts age instead of survival:

In [ ]:
features_reg = ["Pclass", "Fare", "Sex_label", "Cabin_label", "Embarked_label", "Survived"]
target_reg = "Age"

tree_regressor = DecisionTreeRegressor(max_depth=3, random_state=random_state, criterion="absolute_error")
tree_regressor.fit(dataset[features_reg].values, dataset[target_reg].values)

## Initialize dtreeviz model (adaptor)

In [ ]:
viz_rmodel = dtreeviz.model(model=tree_regressor, 
                            X_train=dataset[features_reg], 
                            y_train=dataset[target_reg], 
                            feature_names=features_reg, 
                            target_name=target_reg)

## Tree structure visualisations

In [ ]:
viz_rmodel.view()

In [ ]:
viz_rmodel.view(orientation="LR")

In [ ]:
viz_rmodel.view(fancy=False)

In [ ]:
viz_rmodel.view(depth_range_to_display=(0, 2))

## Prediction path explanations

In [ ]:
x = dataset[features_reg].iloc[10]
x

In [ ]:
viz_rmodel.view(x = x)

In [ ]:
viz_rmodel.view(show_just_path=True, x = x)

In [ ]:
print(viz_rmodel.explain_prediction_path(x))

In [ ]:
viz_rmodel.instance_feature_importance(x, figsize=(3.5,2))

## Leaf info

In [ ]:
viz_rmodel.leaf_sizes(figsize=(3.5,2))

In [ ]:
viz_rmodel.rtree_leaf_distributions()

In [ ]:
viz_rmodel.node_stats(node_id=4)

In [ ]:
viz_rmodel.leaf_purity(figsize=(3.5,2))

# Feature Space Partitioning

Decision trees partition feature space in such a way as to maximize target value purity for the instances associated with a node. It's often useful to visualize the feature space partitioning, although it's not feasible to visualize more than a couple of dimensions.

## Classification

To visualize how it decision tree partitions a single feature, let's train a shallow decision tree classifier using the toy Iris data.

In [ ]:
from sklearn.datasets import load_iris
iris = load_iris()
X = iris.data
class_names = iris.target_names

X = X[:,2].reshape(-1,1) # petal length (cm)
y = iris.target

In [ ]:
dtc_univar = DecisionTreeClassifier(max_depth=2, min_samples_leaf=1)
dtc_univar.fit(X, y)

In [ ]:
viz_model = dtreeviz.model(dtc_univar,
                           X_train=X, y_train=y,
                           feature_names=['petal length (cm)'],
                           target_name='iris',
                           class_names=class_names)

The following diagram indicates that the decision tree splits the petal length feature into three mostly-pure regions:

In [ ]:
viz_model.ctree_feature_space(nbins=40, gtype='barstacked', show={'splits','title'},
                             figsize=(5,2))

A deeper tree gives this finer grand partitioning of the single feature space:

In [ ]:
dtc_univar = DecisionTreeClassifier(max_depth=3, min_samples_leaf=1)
dtc_univar.fit(X, y)

viz_model = dtreeviz.model(dtc_univar,
                           X_train=X, y_train=y,
                           feature_names=['petal length (cm)'],
                           target_name='iris',
                           class_names=class_names)

viz_model.ctree_feature_space(nbins=40, gtype='barstacked', show={'splits','title'}, figsize=(5,2))

Let's look at how a decision tree partitions two-dimensional feature space.

In [ ]:
X = iris.data
X = X[:,[0,3]] # 'sepal length (cm)', 'petal width (cm)'
y = iris.target
dtc_bivar = DecisionTreeClassifier(max_depth=3, min_samples_leaf=1)
dtc_bivar.fit(X, y)

In [ ]:
viz_model = dtreeviz.model(dtc_bivar,
                           X_train=X, y_train=y,
                           feature_names=['sepal length (cm)', 'petal width (cm)'], 
                           target_name='iris',
                           class_names=class_names)

viz_model.ctree_feature_space(nbins=40, gtype='barstacked', show={'splits','title'})

## Regression

To demonstrate regression, let's load a toy Cars data set and visualize the partitioning of univariate and bivariate feature spaces.

In [ ]:
dataset_url = "https://raw.githubusercontent.com/parrt/dtreeviz/master/data/cars.csv"
df_cars = pd.read_csv(dataset_url)
X = df_cars[['WGT']]
y = df_cars['MPG']

In [ ]:
dtr_univar = DecisionTreeRegressor(max_depth=3, criterion="absolute_error")
dtr_univar.fit(X.values, y.values)

In [ ]:
viz_rmodel = dtreeviz.model(dtr_univar, X, y,
                           feature_names=['WGT'],
                           target_name='MPG')

The following visualization illustrates how the decision tree breaks up the `WGT` (car weight) in order to get relatively pure `MPG` (miles per gallon) target values.

In [ ]:
viz_rmodel.rtree_feature_space()

In order to visualize two-dimensional feature space, we can draw in three dimensions.

In [ ]:
X = df_cars[["WGT", "ENG"]]
y = df_cars['MPG']
dtr_bivar_3d = DecisionTreeRegressor(max_depth=3, criterion="absolute_error")
dtr_bivar_3d.fit(X.values, y.values)

In [ ]:
viz_rmodel = dtreeviz.model(dtr_bivar_3d, X, y,
                           feature_names=["WGT", "ENG"],
                           target_name='MPG')

In [ ]:
viz_rmodel.rtree_feature_space3D(fontsize=10,
                        elev=30, azim=20, 
                        show={'splits','title'},
                        colors={'tessellation_alpha':.5})

Equivalently, we can show a heat map as if we were looking at the three-dimensional plot from the top down:

In [ ]:
viz_rmodel.rtree_feature_space()